## 공시정보 - 고유번호 목록 API

DART에 등록되어있는 공시대상회사의 고유번호, 회사명, 대표자명, 종목코드, 최근변경일자를 파일로 가져옵니다.
stockcode.json에 등록되어 있는 데이터를 바탕으로 실제 증시에 상장한 공시대상회사만 선별하여 고유번호(8자리)를 corp_code.json에 저장합니다.

In [ ]:
import requests
import zipfile
from io import BytesIO
import xmltodict
import json

url = "	https://opendart.fss.or.kr/api/corpCode.xml"
crtfc_key =""

r = requests.get(url, params={'crtfc_key': ""})
r.raise_for_status()
z = zipfile.ZipFile(BytesIO(r.content))
print(z.namelist())
data_xml = z.read(z.namelist()[0]).decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

data = [item for item in data if item.get("stock_code") != None]

with open('stockcode.json') as r:
    stockcode = json.load(r)
    
data = [item for item in data if item.get("stock_code") in stockcode.keys()]
print(len(data))
with open("corp_code.json", "w") as w:
    json.dump(data, w, ensure_ascii=False)

## 상장기업 재무정보 - 단일회사 주요계정 API

DART에서 제공하는 상장법인(금융업 제외)이 제출한 정기보고서 내에 XBRL재무제표의 주요계정과목(재무상태표, 손익계산서)의 데이터를 가져옵니다.

In [8]:
import requests
import json
import pandas as pd

corp_code = '00126380'
bsns_year = '2020'
reprt_code = '11012'
crtfc_key = ''
payload = {'crtfc_key': crtfc_key,'corp_code': corp_code, 'bsns_year':bsns_year, 'reprt_code': reprt_code, 'fs_div':'CFS'}
r = requests.get('https://opendart.fss.or.kr/api/fnlttSinglAcnt.json', params=payload)
data_json = json.loads(r.text)
data = data_json.get('list')
df = pd.json_normalize(data)
df.head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,ord,thstrm_add_amount,frmtrm_add_amount
0,20200814001766,11012,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 52 기반기말,2020.06.30 현재,"186,136,845,000,000",제 51 기말,2019.12.31 현재,"181,385,260,000,000",1,NaN,NaN
1,20200814001766,11012,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 52 기반기말,2020.06.30 현재,"171,822,654,000,000",제 51 기말,2019.12.31 현재,"171,179,237,000,000",3,NaN,NaN
2,20200814001766,11012,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 52 기반기말,2020.06.30 현재,"357,959,499,000,000",제 51 기말,2019.12.31 현재,"352,564,497,000,000",5,NaN,NaN
3,20200814001766,11012,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 52 기반기말,2020.06.30 현재,"61,863,721,000,000",제 51 기말,2019.12.31 현재,"63,782,764,000,000",7,NaN,NaN
4,20200814001766,11012,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 52 기반기말,2020.06.30 현재,"26,287,961,000,000",제 51 기말,2019.12.31 현재,"25,901,312,000,000",9,NaN,NaN


## 사업보고서 주요정보 - 증자(감자) 현황

정기보고서(사업, 분기, 반기보고서) 내에 증자(감자) 현황을 제공합니다.

In [9]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'
crtfc_key = ''
payload = {'crtfc_key': crtfc_key,'corp_code': corp_code, 'bsns_year':bsns_year, 'reprt_code': reprt_code}
r = requests.get('https://opendart.fss.or.kr/api/irdsSttus.json', params=payload)
data_json = json.loads(r.text)
data = data_json.get('list')
df = pd.json_normalize(data)
df

,rcept_no,corp_cls,corp_code,corp_name,isu_dcrs_de,isu_dcrs_stle,isu_dcrs_stock_knd,isu_dcrs_qy,isu_dcrs_mstvdv_fval_amount,isu_dcrs_mstvdv_amount
0,20200330003851,Y,00126380,삼성전자,-,-,-,-,-,-


## 공시정보 - 공시서류원본파일

공시보고서 원본파일을 다운로드 받습니다.

In [ ]:
import requests
import zipfile
from io import BytesIO
import xmltodict
import json

url = "https://opendart.fss.or.kr/api/document.xml"

payload = {'rcept_no':'20200330003851', 'crtfc_key': ""}
r = requests.get(url, params=payload)
r.raise_for_status()

z = zipfile.ZipFile(BytesIO(r.content))
print(z.namelist())
file_list = z.namelist()
data_xml = z.read(file_list[0]).decode('cp949')


## 상장기업 재무 정보 - AWS DynamoDB에 저장하기

단일회사 주요계정 API를 사용하여 상장기업의 재무 데이터를 dynamoDB에 저장합니다.

In [12]:
import pandas as pd
import requests
import json
import re
import boto3
from datetime import datetime
import time


def check_value(value):
    regex = '(\d{4}).(\d{2}).(\d{2}) ~ '
    return re.sub(regex, '', str(value))

def request_single_account(payload):
    r = requests.get('https://opendart.fss.or.kr/api/fnlttSinglAcnt.json', params=payload)
    data_json = json.loads(r.text)
    if data_json.get('status') == '013':
        print(data_json.get('message'))
        return ''
    data = data_json.get('list')
    dart = loop_data(data, 'CFS')
    if not dart:
        dart = loop_data(data, 'OFS')
    if not dart:
        return dart
    dart['thstrm_dt'] = check_value(dart.get('thstrm_dt'))
    
    df = pd.json_normalize(dart)
   
    result = df.to_json(orient="records")
    return json.loads(result)

def loop_data(data, fs_div):
    dart = {}
    for item in data:
        if item.get('fs_div') == fs_div:
            account_nm = item.get('account_nm')
            thstrm_amount= item.get('thstrm_amount')
            stock_code = item.get('stock_code')
            thstrm_dt = item.get('thstrm_dt')
            dart[account_nm] = thstrm_amount.replace(',', '')
            dart['stock_code'] = stock_code
            dart['thstrm_dt'] = thstrm_dt
    return dart

def set_payload(corp_code):
    bsns_year = '2015'
    reprt_code = '11011'# 1분기: 11013, 2분기 : 11012, 3분기: 11014, 사업보고서: 11011
    fs_div = 'CFS'
    crtfc_key = ''
    return {'crtfc_key': crtfc_key, 'corp_code': corp_code, 'bsns_year':bsns_year, 'reprt_code': reprt_code, 'fs_div':fs_div}

def view_data(data):
    print(data)
    
def get_corp_code():
    with open('corp_code.json') as r:
        return json.load(r)

def do_work():
    corp_list = get_corp_code()
    for item in corp_list:
#         corp_code = item.get('corp_code')
        corp_code = "00992871"
        print("start : ", corp_code)
        payload = set_payload(corp_code)
        data = request_single_account(payload)
        view_data(data)
        break
        
if __name__ == '__main__':
    do_work()

start :  00992871
{}
[{'유동자산': '288365356022', 'stock_code': '185750', 'thstrm_dt': '2015.12.31', '비유동자산': '287921348405', '자산총계': '576286704427', '유동부채': '151442792128', '비유동부채': '110282845154', '부채총계': '261725637282', '이익잉여금': '19533306789', '자본총계': '314561067145', '매출액': '592493473515', '영업이익': '42705086271', '법인세차감전 순이익': '15137646805', '당기순이익': '-6772741415'}]


In [33]:
def check_value(value):
    regex = '(\d{4}).(\d{2}).(\d{2}) ~ '
    return re.sub(regex, '', str(value))


print(check_value('2019.12.31'))

2019.12.31
